# INSTALL LIBRARIES

In [1]:
# !pip3 install tensorflow_datasets
# !pip3 install pandas

# IMPORT LIBRARIES

In [41]:
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

# Neural Network Components
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

warnings.filterwarnings('always')

In [42]:
from ANN_creator_module import NeuralNetworkConstructor

# READING THE DATA SET

In [51]:
df_path = '/tf/Face_ID1/list_attr_celeba.txt'
df = pd.read_csv(df_path, skiprows=1 ,sep=r'\s+')

In [52]:
# df.head(10)

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
000006.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,1,-1,1,-1,-1,1
000007.jpg,1,-1,1,1,-1,-1,1,1,1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,1
000008.jpg,1,1,-1,1,-1,-1,1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
000009.jpg,-1,1,1,-1,-1,1,1,-1,-1,-1,...,-1,1,-1,-1,1,-1,1,-1,-1,1
000010.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,1


# CREATING THE TENSORFLOW DATASET

In [ ]:
images_dir = '/tf/keras_neural_network/Mis_Tests/img_align_celeba/'
resize_factor = 2
H = 218 // resize_factor
W = 178 // resize_factor
img_size = H  # usar H = W si quieres cuadrado
batch_size = 64
minitrain_size = 2000
minival_size = 1000

In [53]:
df.replace(-1, 0, inplace=True)

removed_columns = [
    'Sideburns','Wearing_Earrings','Mouth_Slightly_Open','Heavy_Makeup',
    'Attractive','Rosy_Cheeks','Wearing_Hat','Wearing_Lipstick',
    'Wearing_Necklace','Wearing_Necktie'
]
df.drop(removed_columns, axis=1, inplace=True)

In [ ]:
label_cols = df.columns
y = df[label_cols].values.astype(np.float32)
paths = (images_dir + df.index).values

In [54]:
idx = np.arange(len(paths))
np.random.seed(42)
np.random.shuffle(idx)

train_idx = idx[:minitrain_size]
val_idx = idx[minitrain_size : minitrain_size + minival_size]

train_paths = paths[train_idx]
train_labels = y[train_idx]

val_paths = paths[val_idx]
val_labels = y[val_idx]

In [55]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (img_size, img_size)) / 255.0
    return img, label

In [56]:
# ====== MINI TRAIN DATASET ======
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

# ====== MINI VAL DATASET ======
val_ds = (
    tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size).prefetch(tf.data.AUTOTUNE)
)

# BUILDING THE MODEL

In [57]:
num_of_classes = 30
inputs = keras.Input((img_size, img_size, 3))
x = inputs
NNC = NeuralNetworkConstructor(x)
conv_outputs = NNC.complete_outputs()
x = layers.GlobalAveragePooling2D()(conv_outputs)
x = layers.Dense(
    units=64, 
    activation='relu', 
    name='Dense1_layer')(x)
outputs = layers.Dense(
    units=num_of_classes, 
    activation='softmax', 
    name='clasification_layer')(x)

In [58]:
model = keras.Model(inputs,outputs)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownConv1 (Conv2D)              │ (None, 126, 126, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownBN1 (BatchNormalization)    │ (None, 126, 126, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownConv2 (Conv2D)              │ (None, 124, 124, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownBN2 (BatchNormalization)    │ (None, 124, 124, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool1 (MaxPooling2D)         │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownDO1 (Dropout)               │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownConv3 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownBN3 (BatchNormalization)    │ (None, 60, 60, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownConv4 (Conv2D)              │ (None, 58, 58, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownBN4 (BatchNormalization)    │ (None, 58, 58, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool2 (MaxPooling2D)         │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DownDO2 (Dropout)               │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense1_layer (Dense)            │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ clasification_layer (Dense)     │ (None, 30)             │         1,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 271,902 (1.04 MB)

 Trainable params: 271,134 (1.03 MB)

 Non-trainable params: 768 (3.00 KB)

# TRAINING THE MODEL

In [59]:
model.compile(
    loss="binary_crossentropy", 
    optimizer=eval(f"keras.optimizers.{'RMSprop'}(learning_rate=0.00388)"), 
    metrics=["accuracy"]
)

In [60]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.0001,
    patience=50, 
    restore_best_weights=True,
    mode="auto",
    verbose=1,
    baseline=None
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=1000, 
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds,
    callbacks=[early_stopping]
)

Epoch 1/1000
 7/32 ━━━━━━━━━━━━━━━━━━━━ 16s 658ms/step - accuracy: 0.0293 - loss: 0.6421